Вот [ТУТ](https://colab.research.google.com/drive/1UeHBfBI34QIOTVylUDEHuvZyQuSXdt25?usp=sharing) взяла еще один сериал -122к примеров, соединила оба датасета и обучала несколько разных моделей, резульаты хорошие

In [ ]:
import os
import re
import nltk
import pandas as pd
import codecs
from nltk.tokenize import sent_tokenize
from tqdm.auto import tqdm
nltk.download('punkt')

!pip install stanza
import stanza
stanza.download('fr') # download French model
nlp_stanza = stanza.Pipeline('fr') # initialize French neural pipeline

!pip install --upgrade spicy
!python -m spacy download fr_core_news_md
import fr_core_news_md
nlp = fr_core_news_md.load()


!pip install benepar
from nltk import Tree
import benepar
benepar.download('benepar_fr2')
import warnings
warnings.filterwarnings("ignore")
import spacy
from benepar.spacy_plugin import BeneparComponent
nlp.add_pipe(BeneparComponent('benepar_fr2'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 286kB 6.3MB/s 


2021-05-05 07:48:17 INFO: Downloading default packages for language: fr (French)...
2021-05-05 07:50:11 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-05-05 07:50:11 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2021-05-05 07:50:12 INFO: Use device: cpu
2021-05-05 07:50:12 INFO: Loading: tokenize
2021-05-05 07:50:12 INFO: Loading: mwt
2021-05-05 07:50:12 INFO: Loading: pos
2021-05-05 07:50:12 INFO: Loading: lemma
2021-05-05 07:50:12 INFO: Loading: depparse
2021-05-05 07:50:13 INFO: Loading: ner
2021-05-05 07:50:14 INFO: Done loading processors!


     |████████████████████████████████| 88.6MB 1.2MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp37-none-any.whl size=90338490 sha256=0779a403abaf2ef24b617aa3954b8af69e337b5c590d6488749188eb587a48a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-g9a3d9fr/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')
     |████████████████████████████████| 3.3MB 9.5MB/s 
     |████████████████████████████████| 2.1MB 50.7MB/s 
     |████████████████████████████████| 1.2MB 41.9MB/s 
     |████████████████████████████████| 901kB 44.1MB/s 
  Created wheel for benepar: filename=benepar-0.2.0-cp37-none-any.whl size=37647 sha256=6f86ce19c8b287082ea8c2ba792c3622ca8af9ffc5376220747d9f976772f6b5
  Stored in directory: /root/.cache/pip/wheels/dd/90/28/513063023646df7774be9401c440a5f13b48bdbab15a67fc42
Successfully built benepar
[

In [ ]:
list_of_paths = []

for root, dirs, files in os.walk("/content/drive/MyDrive/ВКР/proga/DATA/friends"):
    for name in files:
        path = str(root + '/' +  name)
        list_of_paths.append(path)
        #print(os.path.join(root, name))

len(list_of_paths)

145

In [ ]:
def check_unicode(path):
  encodings = ['utf-8', 'windows-1252', 'windows-1250'] # add more
  for e in encodings:
    try:
      fh = codecs.open(path, 'r', encoding=e)
      fh.readlines()
      fh.seek(0)
    except UnicodeDecodeError:
      pass
        #print('got unicode error with %s , trying different encoding' % e)
    else:
      return e 
      break 

In [ ]:
def make_dataframe(list_of_path):
  data = pd.DataFrame()
  sentences = []
  for path in list_of_path:
    text = []
    encoding = check_unicode(path)
    with open(path,'r', encoding=encoding) as f: 
      lines = f.readlines()
    for x in lines:
      if not re.search(r'[a-zA-Z]', x):
        pass
      else:
        x = re.sub(r'\d\d:\d\d:\d\d,\d\d\d --> \d\d:\d\d:\d\d,\d\d\d  X\d:\d\d\d X\d:\d\d\d Y\d:\d\d\d Y\d:\d\d\d', r'', x)
        x = x.replace('...', ' ')
        if len(x) > 1:
          text.append(x.replace('\n',''))
    sentences = sentences +  sent_tokenize(' '.join(text))
  data['sentences'] = sentences
  return data

sentences = make_dataframe(list_of_path = list_of_paths)

In [ ]:
sentences.head(5)

,sentences
0,"Vous pigez pas que pour nous, les baisers ont..."
1,Ben voyons !
2,Sérieux ?
3,Tout se lit déjà dans le premier baiser.
4,Absolument !


In [ ]:
dependence = []
for i, text in tqdm(enumerate(sentences.sentences)):
  doc = nlp_stanza(text)
  list_doc = doc.to_dict()
  dependence.append(list_doc)
sentences['dependence_structure'] = dependence 

sentences.head()

,sentences,dependence_structure
0,"Vous pigez pas que pour nous, les baisers ont...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up..."
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up..."
2,Sérieux ?,"[[{'id': 1, 'text': 'Sérieux', 'lemma': 'série..."
3,Tout se lit déjà dans le premier baiser.,"[[{'id': 1, 'text': 'Tout', 'lemma': 'tout', '..."
4,Absolument !,"[[{'id': 1, 'text': 'Absolument', 'lemma': 'ab..."


In [ ]:
sentences.to_csv('/content/drive/MyDrive/ВКР/proga/DATA/friends_dependence_final.csv', index=False)

In [ ]:
# находим предложения, где есть починительный союз
idxs_sconj = []
for i, text in enumerate(sentences.dependence_structure):
  for sent in text:
    for word in sent:
      try:
        if word['upos'] == 'SCONJ':
          if i not in idxs_sconj:
            idxs_sconj.append(i)
      except:
        continue

In [ ]:
# берем новый датасет только содержащие подчинительные союзы
sconj_sentences = sentences.iloc[idxs_sconj,:] 

In [ ]:
# показывает дерево предложения
def make_tree(text):
  doc = nlp(text)
  sents = list(doc.sents)
  for sent in sents:
    print(sent._.parse_string)
    result = Tree.fromstring(str(sent._.parse_string)).pretty_print()
  pass

# строит структуру составляющих предложения
def make_structure(text):
    doc = nlp(text)
    sents = list(doc.sents)
    result = [sent._.parse_string for sent in sents]
    return result

# парсим структуру составляющих + берем только уровень 1
def parenthetic(string):
    result = []
    stack = []
    for i, c in enumerate(string):
        if c == '(':
            stack.append(i)
        elif c == ')' and stack:
            start = stack.pop()
            if len(stack) == 1:
              result.append(string[start + 1: i].split(' ')[0])
    return result 

def dep_struct(text):
  doc = nlp_stanza(text)
  list_doc = doc.to_dict()
  return list_doc

In [ ]:
trees = []
for text in tqdm(sconj_sentences.sentences):
  trees.append(make_structure(text))

sconj_sentences['tree_structure'] = trees 


In [ ]:
sconj_sentences.to_csv('/content/drive/MyDrive/ВКР/proga/DATA/friends_sconj_sentences.csv', index=False)

In [ ]:
import pandas as pd 
sentences = pd.read_csv('/content/drive/MyDrive/ВКР/proga/DATA/friends_sconj_sentences.csv')
sentences.head()

,sentences,dependence_structure,tree_structure
0,"Vous pigez pas que pour nous, les baisers ont...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (S...
1,On ne dit pas que la 1ère partie ne nous plaît...,"[[{'id': 1, 'text': 'On', 'lemma': 'on', 'upos...",['(SENT (VN (CLS On) (ADV ne) (V dit)) (ADV pa...
2,Mais ce n'est pas pour ca qu'on a acheté les ...,"[[{'id': 1, 'text': 'Mais', 'lemma': 'mais', '...","[""(SENT (COORD (CC Mais) (Sint (VN (CLS ce) (A..."
3,"Le problème, c'est qu'après, même si ca a été ...","[[{'id': 1, 'text': 'Le', 'lemma': 'le', 'upos...","[""(SENT (NP (DET Le) (NC problème)) (PONCT ,) ..."
4,Mais si !,"[[{'id': 1, 'text': 'Mais', 'lemma': 'mais', '...",['(SENT (COORD (CC Mais) (Sint (ADV si))) (PON...


In [ ]:
len(sentences)

5495

In [ ]:
# Добавим еще одну колонку - закодируем структуру составляющих для обучения
# парсим структуру составляющих
def parenthetic_full(string):
    result = []
    stack = []
    for i, c in enumerate(string):
        if c == '(':
            stack.append(i)
        elif c == ')' and stack:
            start = stack.pop()
            result.append((len(stack),string[start + 1: i].split(' ')[0]))
    return sorted(result)

In [ ]:
parenthetic_full(string)

In [ ]:
# 'AAA' - это PAD
tags = ['AAA'] # 55 тегов
for struct in sentences['tree_structure']:
  for s in eval(struct):
    for x in parenthetic_full(s):
      if x[1] not in tags:
        tags.append(x[1])

In [ ]:
len(tags)

55

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
enc = OrdinalEncoder()
X = np.array(tags).reshape(-1, 1)
enc.fit(X)
enc.transform([['SENT']])

array([[39.]])

In [ ]:
# def func(s, n):
#   result = []
#   for x in parenthetic_full(s):
#     try:
#       if x[0] == n:
#         result.append(int(enc.transform([[x[1]]])))
#     except:
#       result.append(int(enc.transform([['PAD']])))
#   return np.array(result)

def func(s, n):
  result = []
  for x in parenthetic_full(s):
    try:
      if x[0] == n:
        result.append(int(enc.transform([[x[1]]])))
    except:
      result.append(int(enc.transform([['PAD']])))
  return result

for struct in list(sentences['tree_structure'])[:10]:
  for s in eval(struct):
    print(func(s,1))

[4, 32, 42, 48]
[4, 32, 42, 48]
[14, 32]
[24, 32, 32, 42, 48]
[14, 32]
[4, 32, 32, 32, 42, 48]
[24, 32, 33, 42, 48]
[4, 24, 32, 33, 42, 48]
[32, 42]
[32, 42]


In [ ]:
# попробуем разделить уровни составляющих по разным колонкам, максимальный уровень - 4
# from keras.preprocessing.sequence import pad_sequences

# zeros_all = []
# ones_all = []
# twos_all = []
# threes_all = []
# fours_all = []
# for struct in tqdm(list(sentences['tree_structure'])):
#   zeros = []
#   ones = []
#   twos = []
#   threes = []
#   fours = []
#   for s in eval(struct):
#     zeros = zeros + func(s,0)
#     ones= ones + func(s,1 )
#     twos = twos + func(s,2)
#     threes = threes + func(s,3)
#     fours = fours + func(s,4)
#   zeros_all.append(zeros)
#   ones_all.append(ones)
#   twos_all.append(twos)
#   threes_all.append(threes)
#   fours_all.append(fours)

    
# sentences['zeros'] = pad_sequences(zeros_all, padding='post')
# sentences['ones']= pad_sequences(ones_all, padding='post')
# sentences['twos']= pad_sequences(twos_all, padding='post')
# sentences['threes'] = pad_sequences(threes_all, padding='post')
# sentences['fours'] = pad_sequences(fours_all, padding='post') 


In [ ]:
pad_sequences(ones_all, padding='post')

array([[ 4, 32, 42, ...,  0,  0,  0],
       [ 4, 32, 42, ...,  0,  0,  0],
       [14, 32,  0, ...,  0,  0,  0],
       ...,
       [32, 32, 42, ...,  0,  0,  0],
       [32, 32, 42, ...,  0,  0,  0],
       [32, 42, 48, ...,  0,  0,  0]], dtype=int32)

In [ ]:
pad_sequences(twos_all, padding='post')

array([[13, 15, 42, ...,  0,  0,  0],
       [ 4, 13, 15, ...,  0,  0,  0],
       [ 9, 40,  0, ...,  0,  0,  0],
       ...,
       [13, 24, 28, ...,  0,  0,  0],
       [ 4, 13, 15, ...,  0,  0,  0],
       [12, 13, 15, ...,  0,  0,  0]], dtype=int32)

In [ ]:
sentences['ones'] = list(pad_sequences(ones_all, padding='post'))
sentences['twos']  = list(pad_sequences(twos_all, padding='post'))

In [ ]:
sentences.head()

,sentences,dependence_structure,tree_structure,target,ones,twos
0,"Vous pigez pas que pour nous, les baisers ont...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (S...,0,"[4, 32, 42, 48, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 15, 42, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,On ne dit pas que la 1ère partie ne nous plaît...,"[[{'id': 1, 'text': 'On', 'lemma': 'on', 'upos...",['(SENT (VN (CLS On) (ADV ne) (V dit)) (ADV pa...,0,"[4, 32, 42, 48, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 13, 15, 40, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Mais ce n'est pas pour ca qu'on a acheté les ...,"[[{'id': 1, 'text': 'Mais', 'lemma': 'mais', '...","[""(SENT (COORD (CC Mais) (Sint (VN (CLS ce) (A...",0,"[14, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[9, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"Le problème, c'est qu'après, même si ca a été ...","[[{'id': 1, 'text': 'Le', 'lemma': 'le', 'upos...","[""(SENT (NP (DET Le) (NC problème)) (PONCT ,) ...",0,"[24, 32, 32, 42, 48, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[13, 15, 17, 22, 40, 43, 0, 0, 0, 0, 0, 0, 0, ..."
4,Mais si !,"[[{'id': 1, 'text': 'Mais', 'lemma': 'mais', '...",['(SENT (COORD (CC Mais) (Sint (ADV si))) (PON...,0,"[14, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[9, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [ ]:
from tqdm.auto import tqdm
insub_structures = [['Ssub', 'PONCT'],['Ssub'],['ADV', 'ADV', 'VN', 'PONCT']] #['COORD', 'PONCT'], ['COORD'],
insub_list = []
for i, row in  tqdm(sentences.iterrows()):
  for sent in eval(row.tree_structure):
    if parenthetic(sent) in insub_structures:
      insub_list.append(i)
len(insub_list)

546

In [ ]:
# посмотрим глазками, то ли это, что нужно
for i in insub_list:
  print(sentences.sentences.loc[i], '-------', sentences.tree_structure.loc[i])
# нужно убрать все предложения с faux, A moins, D'autant,autant, Mieux, Tellement, Surtout   - они нам не подходят тк que.. будет валентностью и заависимо
# S'il te plaît ! - брать или нет? С одной стороны нам подходит, но с другой это очень устоявшаяся штука
# Sauf si -  ?????
# Alors que - ???
#Quand tu m'as livrée à eux ? - вот такие вопросы нам подходят?

# Не понятно, почему нашел вот такие предложения:
# Va voir si tu les as pas laissés à la copieuse. 
# Comme je sais que je vois mon fils mort derrière vous. [[ssub], [ssub]]

Parce que mes parents lui donnent le bon Dieu sans confession ! ------- ['(SENT (Ssub (CS+ (CS Parce) (CS que)) (Sint (NP (DET mes) (NC parents)) (VN (CLO lui) (V donnent)) (NP (DET le) (ADJ bon) (NC Dieu)) (PP (P sans) (NP (NC confession))))) (PONCT !))']
Comme si je ne redoutais pas assez de devoir la lui rendre demain. ------- ['(SENT (Ssub (CS+ (ADV Comme) (CS si)) (Sint (VN (CLS je) (ADV ne) (V redoutais)) (AdP (ADV pas) (ADV assez)) (PP (P de) (VPinf (VN (VINF devoir)) (VPinf (VN (CLO la) (CLO lui) (VINF rendre)) (ADV demain)))))) (PONCT .))']
Que je puisse souffler. ------- ['(SENT (Ssub (CS Que) (Sint (VN (CLS je) (VS puisse)) (VPinf (VN (VINF souffler))))) (PONCT .))']
Si c'est un garcon. ------- ["(SENT (Ssub (CS Si) (Sint (VN (CLS c') (V est)) (NP (DET un) (NC garcon)))) (PONCT .))"]
Parce qu'il est un peu analytique ? ------- ["(SENT (Ssub (CS+ (CS Parce) (CS qu')) (Sint (VN (CLS il) (V est)) (AP (ADV+ (DET un) (ADV peu)) (ADJ analytique)))) (PONCT ?))"]
L'aurais-je épousée

In [ ]:
# итого, учтем все предыдущие фильтры
def make_insub_list(sentences):
  # этот списока, кажется надо дорабатыавть, пока содержит конструкции, которые я заметила. Последняя штука - субжонктив
  insub_structures = [['Ssub', 'PONCT'],['Ssub'],['ADV', 'ADV', 'VN', 'PONCT']] #['COORD', 'PONCT'], ['COORD'],
  exceptions = ['faut', 'a moins' 'autant', 'mieux', 'tellement', 'surtou']
  insub_list = []
  for i, row in  tqdm(sentences.iterrows()):
    for sent in eval(row.tree_structure):
      if parenthetic(sent) in insub_structures:
        if not any(x in row.sentences.lower() for x in exceptions):
          insub_list.append(i)
  return insub_list
insub_list =  make_insub_list(sentences)
len(insub_list)

486

In [ ]:
# берем новый датасет только содержащий расподчинение
insub_sentences = sentences.iloc[insub_list,:] 
insub_sentences.to_csv('/content/drive/MyDrive/ВКР/proga/DATA/insub_corpora/friends.csv', index=False)

In [ ]:
insub_idxs = []
for i in range(len(sentences)):
  if i not in insub_list:
    insub_idxs.append(0)
  else:
    insub_idxs.append(1)

sentences['target'] = insub_idxs 
sentences.head(3)

,sentences,dependence_structure,tree_structure,target
0,"Vous pigez pas que pour nous, les baisers ont...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (S...,0
1,On ne dit pas que la 1ère partie ne nous plaît...,"[[{'id': 1, 'text': 'On', 'lemma': 'on', 'upos...",['(SENT (VN (CLS On) (ADV ne) (V dit)) (ADV pa...,0
2,Mais ce n'est pas pour ca qu'on a acheté les ...,"[[{'id': 1, 'text': 'Mais', 'lemma': 'mais', '...","[""(SENT (COORD (CC Mais) (Sint (VN (CLS ce) (A...",0


# Попробуем что-нибудь обучить на закодированной структуре составляющих

In [ ]:
# для обучения добавим в наш датасет примеры без подчинительного союза
sentences_full = pd.read_csv('/content/drive/MyDrive/ВКР/proga/DATA/friends_dependences_new.csv')
sentences_full 

,sentences,dependence_structure,SCONJ,tree_structure
0,"Vous pigez pas... que pour nous, les baisers o...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",1,['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (P...
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up...",0,['(SENT (ADV Ben) (VN (VIMP voyons)) (PONCT !))']
2,Sérieux ?,"[[{'id': 1, 'text': 'Sérieux', 'lemma': 'série...",0,['(SENT (AP (ADJ Sérieux)) (PONCT ?))']
3,Tout se lit déjà dans le premier baiser.,"[[{'id': 1, 'text': 'Tout', 'lemma': 'tout', '...",0,['(SENT (NP (PRO Tout)) (VN (CLR se) (V lit)) ...
4,Absolument !,"[[{'id': 1, 'text': 'Absolument', 'lemma': 'ab...",0,['(SENT (ADV Absolument) (PONCT !))']
...,...,...,...,...
49807,Ma titine est enfin libre !,"[[{'id': 1, 'text': 'Ma', 'lemma': 'son', 'upo...",0,['(SENT (NP (DET Ma) (NC titine)) (VN (V est))...
49808,Démarre !,"[[{'id': 1, 'text': 'Démarre', 'lemma': 'démar...",0,['(SENT (NP (VIMP Démarre)) (PONCT !))']
49809,On est pas sexy ?,"[[{'id': 1, 'text': 'On', 'lemma': 'on', 'upos...",0,['(SENT (VN (CLS On) (V est)) (ADV pas) (AP (A...
49810,Tu veux acheter une caisse ?,"[[{'id': 1, 'text': 'Tu', 'lemma': 'il', 'upos...",0,['(SENT (VN (CLS Tu) (V veux)) (VPinf (VN (VIN...


In [ ]:
not_sconj = pd.DataFrame()
s_texts = list(sentences.sentences)
for i,row in sentences_full.iterrows():
  if row.sentences not in s_texts:
    if len(eval(row.tree_structure)) == 1:
      not_sconj  = not_sconj.append({'sentences': row.sentences, 'tree_structure':row.tree_structure},ignore_index=True)
  if len(not_sconj) == 10000:
    break
not_sconj.head()

,sentences,tree_structure
0,Ben voyons !,['(SENT (ADV Ben) (VN (VIMP voyons)) (PONCT !))']
1,Sérieux ?,['(SENT (AP (ADJ Sérieux)) (PONCT ?))']
2,Tout se lit déjà dans le premier baiser.,['(SENT (NP (PRO Tout)) (VN (CLR se) (V lit)) ...
3,Absolument !,['(SENT (ADV Absolument) (PONCT !))']
4,"Pour nous, les baisers, c'est une 1ère partie.","[""(SENT (PP (P Pour) (NP (PRO nous))) (PONCT ,..."


In [ ]:
# кодируем структуру дерева и приводим к одному размеру
ones_all = []
for struct in tqdm(list(sentences['tree_structure'])):
  ones = []
  for s in eval(struct):
    ones= ones + func(s,1)
  ones_all.append(ones)

for struct in tqdm(list(not_sconj['tree_structure'])):
  ones = []
  for s in eval(struct):
    ones= ones + func(s,1)
  ones_all.append(ones)

In [ ]:
len(ones_all)

15495

In [ ]:
not_sconj['target'] = [0]*len(not_sconj)

In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
def scores(model):
    print('Accuracy: ', round(model.score(X_test, y_test),4))
    # y_score = model.decision_function(X_test)
    # average_precision = average_precision_score(y_test, y_score)
    # print(f'Precision: {average_precision}')
    y_pred = model.predict(X_test)
    recall = recall_score(y_test, y_pred)
    print(f'Recall: {recall}')
    f1 = f1_score(y_test, y_pred)
    print(f'F1 score: {round(f1,4)}')
    average_precision = average_precision_score(y_test, y_pred)
    print(f'Precision: {average_precision}')

In [ ]:
X = pad_sequences(ones_all, padding='post') 
y = np.array(sentences['target'].append(not_sconj['target']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y, shuffle=True)

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
scores(neigh)

Accuracy:  0.9908


In [ ]:
X

array([[ 4, 32, 42, ...,  0,  0,  0],
       [ 4, 32, 42, ...,  0,  0,  0],
       [14, 32,  0, ...,  0,  0,  0],
       ...,
       [24, 32, 33, ...,  0,  0,  0],
       [24, 32, 48, ...,  0,  0,  0],
       [24, 32, 48, ...,  0,  0,  0]], dtype=int32)

In [ ]:
scores(neigh)

Accuracy:  0.9908
Recall: 0.9371069182389937
F1 score: 0.8638
Precision: 0.7526485929424791


In [ ]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred =  neigh.predict(X_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4955
           1       0.80      0.94      0.86       159

    accuracy                           0.99      5114
   macro avg       0.90      0.96      0.93      5114
weighted avg       0.99      0.99      0.99      5114



In [1]:
class_names = ['non-insubordination', 'insubordination']
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

# Plot non-normalized confusion matrix
titles_options = [("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(neigh, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

NameError: ignored